# Understanding the PushBlock case
___
## 1. Introduction
In this notebook we analyse one of the examples in the ml-agents repository, the PushBlock example. 
Also we will analyse its performance with several parameter changes. And in the end we will take a look at a slightly modified version of the PushBlock case and see how it works compared to the original one.

The team responsible for this work is formed by:  

<img src="adria_foto.jpg" style="width: 200px; margin: 10px 0px;  border: 1px solid black;"> 
Adrià Ortiz Navarro  
correu@gmail.com  
<img src="ferran_foto.jpg" style="width: 200px; margin: 10px 0px;  border: 1px solid black;"> 
Ferran Illa Capellas  
ferran.illa26@gmail.com




## 2. Case analysis
a) what are the rewards  
b) what are the states  
c) how is the training implemented

## 3. Performance analysis

## 4. New case proposal